# Mistral

# Lesson 6: Essay Writer

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

In [3]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [4]:
#from langchain_openai import ChatOpenAI
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

from langchain_mistralai.chat_models import ChatMistralAI
model = ChatMistralAI(model="mistral-large-latest", temperature=0)


In [5]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

In [6]:
WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [7]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

In [8]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""


In [9]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [10]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [11]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [12]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

In [13]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [14]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [15]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [16]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [17]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [18]:
builder = StateGraph(AgentState)

In [19]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [20]:
builder.set_entry_point("planner")

In [21]:
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)


In [22]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [23]:
graph = builder.compile(checkpointer=memory)

In [24]:
from IPython.display import Image

Image(graph.get_graph().draw_png())

ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.

In [25]:
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    'task': "what is the difference between langchain and langsmith",
    "max_revisions": 2,
    "revision_number": 1,
}, thread):
    print(s)

{'planner': {'plan': "Title: Langchain vs. Langsmith: A Comparative Analysis\n\nI. Introduction\n   A. Brief overview of the essay's purpose\n   B. Importance of understanding the differences between Langchain and Langsmith\n   C. Brief introduction to Langchain and Langsmith\n\nII. Background Information\n   A. Explanation of what Langchain is\n      1. Developer and release date\n      2. Purpose and functionality\n   B. Explanation of what Langsmith is\n      1. Developer and release date\n      2. Purpose and functionality\n\nIII. Key Features Comparison\n   A. Detailed comparison of the key features of Langchain\n      1. Unique features and their benefits\n      2. Practical applications\n   B. Detailed comparison of the key features of Langsmith\n      1. Unique features and their benefits\n      2. Practical applications\n\nIV. Performance and Efficiency\n   A. Comparison of Langchain's performance and efficiency\n      1. Speed, accuracy, and reliability\n      2. User experie

{'generate': {'draft': "**Title: Langchain vs Langsmith: Contrasting AI Language Model Frameworks**\n\nI. Introduction\nIn the realm of AI language model frameworks, Langchain and Langsmith stand out as prominent tools. While both serve the purpose of enabling applications with large language models, they possess unique characteristics that set them apart. This essay delves into the disparities between Langchain and Langsmith to provide a comprehensive understanding of their functionalities and implications.\n\nII. Langchain\nLangchain is an open-source framework designed to facilitate the development of applications utilizing large language models like GPT-3. It offers versatility in building projects and is not limited to a specific programming language. Key features include seamless integration with various tools, scalability, and a wide range of applications in different industries. However, Langchain may have limitations in terms of complexity for beginners and specific use cases.

## Essay Writer Interface

In [27]:
import warnings
warnings.filterwarnings("ignore")

from helper import ewriter, writer_gui

ModuleNotFoundError: No module named 'langchain_openai'

In [28]:
"""
MultiAgent = ewriter()
app = writer_gui(MultiAgent.graph)
app.launch()
"""

'\nMultiAgent = ewriter()\napp = writer_gui(MultiAgent.graph)\napp.launch()\n'